<a href="https://colab.research.google.com/github/lgiesen/forest_height/blob/main/notebooks/generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [4]:
from google.colab import drive
drive.mount ('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# the zipped data is uploaded in the root_path folder
root_path = 'drive/MyDrive/Colab Notebooks/data/'
path_images = f'{root_path}images/'
path_masks = f'{root_path}masks/'
user = "lgiesen"
repo = "forest_height"
!git clone https://github.com/{user}/{repo}.git

Cloning into 'forest_height'...
remote: Enumerating objects: 563, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 563 (delta 114), reused 202 (delta 109), pack-reused 348
Receiving objects: 100% (563/563), 38.49 MiB | 22.20 MiB/s, done.
Resolving deltas: 100% (305/305), done.


In [6]:
%run /content/forest_height/src/generate_data_VIs.py

Exception: ignored

In [ ]:
%%time
# unzip data
%cd "drive/MyDrive/Colab Notebooks/data/"
# use -B flag to rename files if there is a file with its name
!for f in *.zip; do unzip -B "$f"; done
%cd ../../../../

In [4]:
def save_dataset(X_train, X_test, y_train, y_test):
  X_train.to_pickle("X_train.pkl")
  y_train.to_pickle("y_train.pkl")
  X_test.to_pickle("X_test.pkl")
  y_test.to_pickle("y_test.pkl")
  %cd ../../

Make directories for all types of datasets. All datasets are not upsampled here. To make them upsampled, set the parameter `is_balanced=True`.

In [5]:
%cd forest_height/data/
!mkdir color_channels color_channels_ndvi ndvi vi

/content/forest_height/data
mkdir: cannot create directory ‘color_channels’: File exists
mkdir: cannot create directory ‘color_channels_ndvi’: File exists
mkdir: cannot create directory ‘ndvi’: File exists


Only color channels

In [67]:
%cd ..
X_train, X_test, y_train, y_test = generate_dataset(path_images, path_masks, ['color_channels'])
%cd forest_height/data/color_channels
save_dataset(X_train, X_test, y_train, y_test)


/content
/content/forest_height/data/color_channels
/content/forest_height


Color channels and ndvi value

In [68]:
%cd ..
X_train, X_test, y_train, y_test = generate_dataset(path_images, path_masks, ['color_channels', 'NDVI'])
%cd forest_height/data/color_channels_ndvi
save_dataset(X_train, X_test, y_train, y_test)

/content
/content/forest_height/data/color_channels_ndvi
/content/forest_height


Only ndvi value

In [69]:
%cd ..
X_train, X_test, y_train, y_test = generate_dataset(path_images, path_masks, ['NDVI'])
%cd forest_height/data/ndvi
save_dataset(X_train, X_test, y_train, y_test)

/content
/content/forest_height/data/ndvi
/content/forest_height


In [70]:
%cd ..
X_train, X_test, y_train, y_test = generate_dataset(path_images, path_masks, ['VI'])
%cd forest_height/data/vi
save_dataset(X_train, X_test, y_train, y_test)

/content
/content/forest_height/data/vi
/content/forest_height


In [ ]:
# remove drive connection as it is no longer needed
drive.flush_and_unmount()